In [1]:
!pip install pyDOE

#you need to add a dataset with GaussJacobiQuadRule_V3 on the right 
import sys
sys.path.insert(1, '/kaggle/input/quadrule')
from GaussJacobiQuadRule_V3 import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights

#import GaussJacobiQuadRule_V3


import tensorflow as tf
print(tf.version)
#tf.disable_v2_behavior()
import pyDOE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pyDOE import lhs
#from GaussJacobiQuadRule_V3.py import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights
import time

2023-09-22 23:05:50.345589: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-22 23:05:50.355744: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 23:05:50.435162: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 23:05:50.436424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 23:05:51.235271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<module 'tensorflow._api.v2.version' from '/home/mariano/Documenti/test/base/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [2]:

# Suppose you have a list of dictionaries
data = [
    {'feature1': [1, 2], 'feature2': [3, 4, 5]},
    {'feature1': [5, 6], 'feature2': [7, 8, 5]}
]

# Convert the list of dictionaries into a dictionary of tensors
# This assumes all the dictionary values have the same shape
data_dict = {
    key: tf.constant([d[key] for d in data])
    for key in data[0].keys()
}

# Create a dataset from the dictionary
dataset = tf.data.Dataset.from_tensor_slices(data_dict)

# You can now use the dataset for further processing
for element in dataset:
    print(element["feature2"])



#make this with your features




tf.Tensor([3 4 5], shape=(3,), dtype=int32)
tf.Tensor([7 8 5], shape=(3,), dtype=int32)


In [39]:
def data_generator():
    for e in range(5):
        x_quad_element = tf.constant([1, 2, 3], dtype=tf.float32)  # Translation of the element as a TensorFlow tensor
        data_dict = {
            'x_quad_element': x_quad_element,
        }
        yield data_dict

# Create a dataset from the generator
dataset = tf.data.Dataset.from_generator(data_generator, output_signature={
    'x_quad_element': tf.TensorSpec(shape=(3,), dtype=tf.float32)
})

# Iterate through the dataset and print 'x_quad_element' for each element
for element in dataset:
    print(element['x_quad_element'].numpy())

[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]


In [3]:
class parameters:
    def __init__(self,LR,Opt_Niter,N_Element,N_testfcn,N_Quad,lossb_weight,a,b):
        self.LR = LR
        self.Opt_Niter = Opt_Niter + 1
        self.Opt_tresh = 2e-32
        self.var_form=2
        self.N_Element = N_Element
        self.N_testfcn = N_testfcn #number of test function
        self.N_Quad = N_Quad    #quadrature points 
        self.lossb_weight = lossb_weight #hyprparameter we may turn it up

        #params for the exact sol and boundary term
        self.omega = 8*np.pi
        self.amp = 1
        self.r1 = 80

        #ingredients for loss
        [self.x_quad, self.w_quad] = GaussLobattoJacobiWeights(N_Quad, 0, 0)
        

        self.test_quad_element = self.Test_fcn(self.N_testfcn, self.x_quad)
        self.d1test_quad_element, self.d2test_quad_element = self.dTest_fcn(self.N_testfcn, self.x_quad)


        [self.x_l, self.x_r] = [a, b]     #modify in the future if you want general (a,b) interval
        self.delta_x = (self.x_r - self.x_l)/N_Element
        self.grid = np.asarray([ self.x_l + i*self.delta_x for i in range(self.N_Element+1)])

        #build right side (it doesnt depend on the network)

        self.F_ext_total = []
        for e in range(self.N_Element):
            x_quad_element = self.grid[e] + (self.grid[e+1]-self.grid[e])/2*(self.x_quad+1)  #traslation of the element 

            jacobian = (self.grid[e+1]-self.grid[e])/2
            testfcn_element = np.asarray([ self.Test_f(n,self.x_quad)  for n in range(1, self.N_testfcn+1)])

            #this is fh calculated on the real nodal values summed over each real nodal values(which comes form CGL nodes )

            f_quad_element = self.f_ext(x_quad_element)
            F_ext_element  = jacobian*np.asarray([sum(self.w_quad*f_quad_element*testfcn_element[i]) for i in range(self.N_testfcn)])    
            F_ext_element = F_ext_element[:,None] 
            self.F_ext_total.append(F_ext_element)

        self.F_ext_total = np.asarray(self.F_ext_total)


        self.X_bound = np.asarray([a,b],dtype=np.float32)[:,None]
        self.u_bound   = self.u_exact(self.X_bound)

    def u_exact(self,x):
        utemp = 0.1*np.sin(self.omega*x) + np.tanh(self.r1*x)
        return self.amp*utemp

    def f_ext(self,x):
        gtemp =  -0.1*(self.omega**2)*np.sin(self.omega*x) - (2*self.r1**2)*(np.tanh(self.r1*x))/((np.cosh(self.r1*x))**2)
        return -self.amp*gtemp

    def Test_f(self,n,x):
        test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
        return test
    
    def Test_fcn(self, N_test,x):
        test_total = []
        for n in range(1,N_test+1):  
            test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
            test_total.append(test)
        return np.asarray(test_total)

    def dTest_fcn(self, N_test,x):  #valuete the first and second derivatives of test functions on a point x 
        d1test_total = []
        d2test_total = []
        for n in range(1,N_test+1):  
            if n==1:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            elif n==2:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
            else:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
        return np.asarray(d1test_total), np.asarray(d2test_total)

In [5]:
float_type_tf=tf.float32
float_type_np=np.float32

In [22]:
params=parameters(0.001,5000,5,20,100,1,-1,1)

print(params.F_ext_total)

[[[-1.42151662e+00]
  [-1.01002384e+00]
  [-7.64583410e+00]
  [ 2.63004706e+00]
  [ 5.69885964e+00]
  [-9.89962193e-01]
  [-1.32092296e+00]
  [ 1.56986039e-01]
  [ 1.53116698e-01]
  [-1.39251152e-02]
  [-1.07496275e-02]
  [ 7.94073297e-04]
  [ 5.08264346e-04]
  [-3.16575955e-05]
  [-1.73255019e-05]
  [ 9.33625645e-07]
  [ 4.46546128e-07]
  [-2.12149474e-08]
  [-9.01475844e-09]
  [ 3.83081587e-10]]

 [[ 8.78545588e-01]
  [ 2.64427675e+00]
  [ 4.72538535e+00]
  [-6.88555258e+00]
  [-3.52208895e+00]
  [ 2.59175467e+00]
  [ 8.16375286e-01]
  [-4.10994785e-01]
  [-9.46313239e-02]
  [ 3.64564249e-02]
  [ 6.64363514e-03]
  [-2.07891088e-03]
  [-3.14124641e-04]
  [ 8.28806611e-05]
  [ 1.07077491e-05]
  [-2.44426361e-06]
  [-2.75980646e-07]
  [ 5.55414765e-08]
  [ 5.57144268e-09]
  [-1.00290728e-09]]

 [[-1.49496924e-13]
  [-2.80276643e+01]
  [ 1.75312727e-13]
  [ 4.11939325e+01]
  [-2.03896839e-13]
  [-4.11963666e+01]
  [ 2.34278178e-13]
  [ 4.19948132e+01]
  [-2.76401037e-13]
  [-4.35696828e+

/home/mariano/Documenti/test/base/lib/python3.9/site-packages/scipy/special/_orthogonal.py:133: RuntimeWarning: invalid value encountered in multiply
  np.poly1d.__init__(self, poly.coeffs * float(kn))


In [51]:

def data_generator():
    for e in range(params.N_Element):

        F_ext_element=tf.constant(params.F_ext_total[e], dtype=tf.float32)
        Ntest_element=tf.constant(np.shape(F_ext_element)[0])
        x_quad_element = tf.constant(params.grid[e] + (params.grid[e+1]-params.grid[e])/2*(params.x_quad+1), dtype=tf.float32)  # Translation of the element as a TensorFlow tensor
        jacobian=tf.constant((params.grid[e+1]-params.grid[e])/2,dtype=tf.float32)
        x_b_element=tf.constant(np.array([[params.grid[e]], [params.grid[e+1]]]),dtype=tf.float32)
        test_quad_element=tf.constant(params.Test_fcn(Ntest_element, params.x_quad),dtype=tf.float32)
        d1test_quad_element, d2test_quad_element = tf.constant(params.dTest_fcn(Ntest_element, params.x_quad),dtype=tf.float32)
        d1test_bound_element, d2test_bound_element = tf.convert_to_tensor(params.dTest_fcn(Ntest_element, np.array([[-1],[1]])),tf.float32)


        data_dict = {
            "F_ext_element":F_ext_element,  
            "x_quad_element":x_quad_element,
            "jacobian":jacobian,
            "x_b_element":x_b_element,
            "test_quad_element":test_quad_element,
            "d1test_quad_element":d1test_quad_element,
            "d2test_quad_element":d2test_quad_element,
            "d1test_bound_element":d1test_bound_element,
            "d2test_bound_element":d2test_bound_element
                    }
        yield data_dict


# Create a dataset from the generator
dataset = tf.data.Dataset.from_generator(data_generator, output_signature={
    "F_ext_element":tf.TensorSpec(shape=(params.N_testfcn,1), dtype=tf.float32),
    'x_quad_element': tf.TensorSpec(shape=(params.N_Quad,), dtype=tf.float32),
    "jacobian":tf.TensorSpec(shape=(), dtype=tf.float32),
    "x_b_element":tf.TensorSpec(shape=(2,1), dtype=tf.float32),
    "test_quad_element":tf.TensorSpec(shape=(params.N_testfcn,params.N_Quad), dtype=tf.float32),
    "d1test_quad_element":tf.TensorSpec(shape=(params.N_testfcn,params.N_Quad), dtype=tf.float32),
    "d2test_quad_element":tf.TensorSpec(shape=(params.N_testfcn,params.N_Quad), dtype=tf.float32),
    "d1test_bound_element":tf.TensorSpec(shape=(params.N_testfcn,2,1), dtype=tf.float32),
    "d2test_bound_element":tf.TensorSpec(shape=(params.N_testfcn,2,1), dtype=tf.float32),
})

# Iterate through the dataset and print 'x_quad_element' for each element
for element in dataset:
    print(element["F_ext_element"])



tf.Tensor(
[[-1.4215167e+00]
 [-1.0100238e+00]
 [-7.6458340e+00]
 [ 2.6300471e+00]
 [ 5.6988597e+00]
 [-9.8996222e-01]
 [-1.3209230e+00]
 [ 1.5698604e-01]
 [ 1.5311670e-01]
 [-1.3925116e-02]
 [-1.0749628e-02]
 [ 7.9407328e-04]
 [ 5.0826435e-04]
 [-3.1657597e-05]
 [-1.7325501e-05]
 [ 9.3362564e-07]
 [ 4.4654612e-07]
 [-2.1214948e-08]
 [-9.0147587e-09]
 [ 3.8308159e-10]], shape=(20, 1), dtype=float32)
tf.Tensor(
[[ 8.7854558e-01]
 [ 2.6442769e+00]
 [ 4.7253852e+00]
 [-6.8855524e+00]
 [-3.5220890e+00]
 [ 2.5917547e+00]
 [ 8.1637532e-01]
 [-4.1099480e-01]
 [-9.4631322e-02]
 [ 3.6456425e-02]
 [ 6.6436352e-03]
 [-2.0789108e-03]
 [-3.1412463e-04]
 [ 8.2880659e-05]
 [ 1.0707749e-05]
 [-2.4442636e-06]
 [-2.7598065e-07]
 [ 5.5541477e-08]
 [ 5.5714429e-09]
 [-1.0029073e-09]], shape=(20, 1), dtype=float32)
tf.Tensor(
[[-1.4949692e-13]
 [-2.8027664e+01]
 [ 1.7531273e-13]
 [ 4.1193932e+01]
 [-2.0389684e-13]
 [-4.1196365e+01]
 [ 2.3427817e-13]
 [ 4.1994812e+01]
 [-2.7640103e-13]
 [-4.3569683e+01]
 [ 

In [35]:
@tf.function
def grad(x):
    with tf.GradientTape() as tape_:
        tape_.watch(x)
        with tf.GradientTape() as tape:
            tape.watch(x)
            y = model(x)
        d1=tape.gradient(y,x)
    d2=tape_.gradient(d1,x)
    return d2,d1

In [70]:
@tf.function
def calculate_loss():
 varloss_total = 0.0
 U_NN_element=0.0

 for element in dataset:

    d2u_NN_quad_element,d1u_NN_quad_element=grad(element["x_quad_element"])



    if params.var_form == 1:
        U_NN_element = tf.reshape(tf.stack([-element["jacobian"]*tf.reduce_sum(params.w_quad*d2u_NN_quad_element*element["test_quad_element"][i]) \
                                        for i in range(params.N_testfcn)]),(-1,1))
    
        
    if params.var_form == 2:
        U_NN_element = tf.reshape(tf.stack([tf.reduce_sum(params.w_quad*d1u_NN_quad_element*element["d1test_quad_element"][i]) \
                                        for i in range(params.N_testfcn)]),(-1,1)) 

    if params.var_form == 3:
        u_NN_bound_element=model(element["x_b_element"])
        u_NN_quad_element=model(element["x_quad_element"])
        U_NN_element = tf.reshape(tf.stack([-1/element["jacobian"]*tf.reduce_sum(params.wquad*u_NN_quad_element*element["d2test_quad_element"][i]) \
                                            +1/element["jacobian"]*tf.reduce_sum(u_NN_bound_element*np.array([-element["d1test_bound_element"][i][0], element["d1test_bound_element"][i][-1]]))  \
                                            for i in range(element["Ntest_element"])]),(-1,1))
        
        
    Res_NN_element = U_NN_element - element["F_ext_element"]
    loss_element = tf.reduce_mean(tf.square(Res_NN_element))
    varloss_total = varloss_total + loss_element




 lossb = tf.reduce_mean(tf.square(model(params.X_bound) - params.u_bound))   #u_NN_pred is what your network has calc,while u_tf is the real value
 return lossb,varloss_total

In [52]:
tfk=tf.keras
tfkl=tf.keras.layers
input_shape=(1,)
seed=53
input_layer = tfkl.Input(shape=input_shape, name='Input')
x1 = tfkl.Dense(20, activation=tf.sin, kernel_initializer = tfk.initializers.GlorotUniform(seed))(input_layer)
x2 = tfkl.Dense(20, activation=tf.sin, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x1)
x3 = tfkl.Dense(20, activation=tf.sin, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x2)
x3 = tfkl.Dense(20, activation=tf.sin, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x2)
output_layer = tfkl.Dense(1, activation='linear', kernel_initializer = tfk.initializers.GlorotUniform(seed))(x3)

In [53]:
model = tf.keras.Model(input_layer, output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 1)]               0         
                                                                 
 dense (Dense)               (None, 20)                40        
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 901 (3.52 KB)
Trainable params: 901 (3.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
epochs=1000
total_record=[]
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

start_time       = time.time()
for epoch in range(epochs+1):
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        loss_valueb,loss_valuev=calculate_loss()
        loss=loss_valueb*params.lossb_weight+loss_valuev
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Print training loss for this epoch
    if (epoch) % 100 == 0:
                elapsed = time.time() - start_time
                str_print = 'It: %d, Lossb: %.3e, Lossv: %.3e, Time: %.2f'
                print(str_print % (epoch, loss_valueb, loss_valuev, elapsed))
                start_time = time.time()        
                total_record.append(np.array([epoch, loss,loss_valueb,loss_valuev]))

ciao
ciao
It: 0, Lossb: 1.204e+00, Lossv: 8.660e+02, Time: 1.10
It: 100, Lossb: 2.879e-05, Lossv: 8.660e+02, Time: 26.64


In [ ]:
x_test=tf.convert_to_tensor(np.linspace(-1,1,100),tf.float32)
y_test=tf.convert_to_tensor(params.u_exact(x_test))


u_pred=model(x_test)
y_test=tf.expand_dims(y_test,axis=1)

fig = plt.figure(0)
gridspec.GridSpec(3,1)


# plt.savefig('Train-Quad-pnts.pdf')    
#++++++++++++++++++++++++++++

font = 24
#all loss
fig, ax = plt.subplots()
plt.tick_params(axis='y', which='both', labelleft='on', labelright='off') 
plt.xlabel('$iteration$', fontsize = font)
plt.ylabel('$loss \,\, values$', fontsize = font)
plt.yscale('log')
plt.grid(True)
iteration = [total_record[i][0] for i in range(len(total_record))]
loss_his  = [total_record[i][1] for i in range(len(total_record))]
plt.plot(iteration, loss_his,'green',label="loss")
plt.legend(loc="upper right")
plt.tick_params( labelsize = 20)
fig.set_size_inches(w=11,h=5.5)
plt.show()
# plt.savefig('loss.pdf')
#++++++++++++++++++++++++++++
#partial loss
fig, ax = plt.subplots()
plt.tick_params(axis='y', which='both', labelleft='on', labelright='off')
plt.xlabel('$iteration$', fontsize = font)
plt.ylabel('$loss \,\, values$', fontsize = font)
plt.yscale('log')
plt.grid(True)
iteration = [total_record[i][0] for i in range(len(total_record))]
loss_his  = [total_record[i][1] for i in range(len(total_record))]
loss_b  = [total_record[i][2] for i in range(len(total_record))]
loss_v  = [total_record[i][3] for i in range(len(total_record))]
plt.plot(iteration, loss_his,'green',label="loss")
plt.plot(iteration, loss_b,'blue',label="boundary_loss")
plt.plot(iteration, loss_v, 'violet',label="variational_loss")
plt.legend(loc="upper right")
plt.tick_params( labelsize = 20)
fig.set_size_inches(w=11,h=5.5)
plt.show()


pnt_skip = 25
fig, ax = plt.subplots()
plt.locator_params(axis='x', nbins=6)
plt.locator_params(axis='y', nbins=8)

plt.xlabel('$x$', fontsize = font)
plt.ylabel('$u$', fontsize = font)
plt.axhline(0, linewidth=0.8, linestyle='-', color='gray')
for xc in params.grid:
    plt.axvline(x=xc, linewidth=2, ls = '--')
plt.plot(x_test, y_test, linewidth=1, color='r', label=''.join(['$exact$']))
plt.plot(x_test, u_pred, 'k*', label='$VPINN$')
plt.tick_params( labelsize = 20)
legend = plt.legend(shadow=True, loc='upper left', fontsize=18, ncol = 1)
fig.set_size_inches(w=11,h=5.5)
plt.show()
# plt.savefig('prediction.pdf')
#++++++++++++++++++++++++++++

fig, ax = plt.subplots()
plt.locator_params(axis='x', nbins=6)
plt.locator_params(axis='y', nbins=8)
plt.xlabel('$x$', fontsize = font)
plt.ylabel('point-wise error', fontsize = font)
plt.yscale('log')
plt.axhline(0, linewidth=0.8, linestyle='-', color='gray')
for xc in params.grid:
    plt.axvline(x=xc, linewidth=2, ls = '--')
plt.plot(x_test, abs(y_test - u_pred), 'k')
plt.tick_params( labelsize = 20)
fig.set_size_inches(w=11,h=5.5)
plt.show()